In [ ]:
!pip install tqdm_joblib
!pip install numpy==1.26.4
!pip install pandas==2.2.2
!pip install scikit-learn==1.3.2
!pip install catboost==1.2.7

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.under_sampling import RandomUnderSampler

# tqdm 관련 임포트 (진행 상황 표시용)
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib

# 0. 문자열 → 숫자로 변환하는 함수
def convert_count_str(val):
    if pd.isna(val):
        return 0.0
    val = str(val).strip()
    if "회 이상" in val:
        return 6.0
    m = re.search(r'(\d+)회?', val)
    if m:
        return float(m.group(1))
    return 0.0

# 기증자 나이 변환을 위한 매핑
donor_age_mapping = {
    '만20세 이하': 0, '만21-25세': 1, '만26-30세': 2, '만31-35세': 3,
    '만36-40세': 4, '만41-45세': 5, '알 수 없음': 0
}
def convert_donor_age(val):
    if pd.isna(val):
        return np.nan
    return donor_age_mapping.get(str(val).strip(), np.nan)

# 카테고리형 변수들의 NaN을 문자열 'NaN'으로 변환하는 함수
def convert_nan_to_string(df, category_columns):
    df_copy = df.copy()
    for col in category_columns:
        df_copy[col] = df_copy[col].fillna('NaN')
    return df_copy

# 카테고리 변수 인덱스 반환 함수 (CatBoost에 사용)
def get_categorical_feature_indices(df):
    cat_features = []
    for idx, (column, dtype) in enumerate(df.dtypes.items()):
        # pandas의 category dtype은 dtype.name == 'category'로 확인
        if dtype.name == 'category':
            cat_features.append(idx)
    return cat_features

# 1. 데이터 로드 및 전처리
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

# 가중치 데이터 로드
weight_data = pd.read_csv('adjusted_weights.csv')
weight_dict = weight_data.set_index("데이터 항목").to_dict()

# '시술 당시 나이' 결측치 여부 추가 (알 수 없음인 경우)
train['시술 당시 나이_missing'] = train['시술 당시 나이'].apply(lambda x: 1.0 if str(x).strip() == '알 수 없음' else 0.0)
test['시술 당시 나이_missing'] = test['시술 당시 나이'].apply(lambda x: 1.0 if str(x).strip() == '알 수 없음' else 0.0)

# '시술 당시 나이' 변환
age_mapping = {
    '만18-34세': 0, '만35-37세': 1, '만38-39세': 2, '만40-42세': 3, '만43-44세': 4, '만45-50세': 5, '알 수 없음': np.nan
}
train['시술 당시 나이'] = train['시술 당시 나이'].apply(lambda x: float(age_mapping.get(str(x).strip(), 0)))
test['시술 당시 나이'] = test['시술 당시 나이'].apply(lambda x: float(age_mapping.get(str(x).strip(), 0)))

# 횟수 관련 컬럼 변환
count_columns = ["총 임신 횟수", "총 출산 횟수", "총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수", "클리닉 내 총 시술 횟수"]
for col in count_columns:
    train[col] = train[col].astype(str).apply(convert_count_str)
    test[col] = test[col].astype(str).apply(convert_count_str)

# 난자/정자 기증자 나이 변환
train['난자 기증자 나이'] = train['난자 기증자 나이'].astype(str).apply(convert_donor_age)
test['난자 기증자 나이'] = test['난자 기증자 나이'].astype(str).apply(convert_donor_age)
train['정자 기증자 나이'] = train['정자 기증자 나이'].astype(str).apply(convert_donor_age)
test['정자 기증자 나이'] = test['정자 기증자 나이'].astype(str).apply(convert_donor_age)

# 2. 가중치 적용 함수
def apply_feature_weights(X, weight_dict):
    X_weighted = X.copy()
    for column in X.columns:
        if column in weight_dict["IVF"]:
            X_weighted[column] *= weight_dict["IVF"][column]  # IVF 가중치 적용
    return X_weighted

# Feature 가중치 적용
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

X_weighted = apply_feature_weights(X, weight_dict)
X_test_weighted = apply_feature_weights(test, weight_dict)

# 3. 데이터 불균형 처리 (임신 성공 여부 기준 언더샘플링)
undersample = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_resampled, y_resampled = undersample.fit_resample(X_weighted, y)

# 4. 데이터 타입 변환
category_columns = [
    "시술 시기 코드", "시술 유형", "특정 시술 유형", "배란 유도 유형",
    "배아 생성 주요 이유", "IVF 임신 횟수", "DI 임신 횟수",
    "IVF 출산 횟수", "DI 출산 횟수", "난자 출처", "정자 출처"
]

# NaN을 문자열로 변환 후 카테고리형으로 변환
X_resampled = convert_nan_to_string(X_resampled, category_columns)
X_test_weighted = convert_nan_to_string(X_test_weighted, category_columns)

for col in category_columns:
    X_resampled[col] = X_resampled[col].astype("category")
    X_test_weighted[col] = X_test_weighted[col].astype("category")

# 5. 학습/검증 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# 6. StackingClassifier를 이용한 모델 앙상블 구성
stack_clf = StackingClassifier(
    estimators=[
        ('xgb', XGBClassifier(
            tree_method='hist',
            device='cuda',
            enable_categorical=True,
            n_estimators=100,      # 트리 개수
            max_depth=6,           # 트리 깊이 제한
            learning_rate=0.1,     # 기본 학습률
            n_jobs=1,              # GPU 사용 시 CPU 스레드 최소화
            use_label_encoder=False,
            eval_metric='logloss'
        )),
        ('lgbm', LGBMClassifier(
            n_jobs=-1,             # CPU 기반 모델이므로 모든 코어 사용 (필요시 적절히 제한 가능)
            verbose=0,
            n_estimators=100,      # 트리 개수
            learning_rate=0.1      # 기본 학습률
        )),
        ('cat', CatBoostClassifier(
            task_type='GPU',
            verbose=0,
            iterations=100,        # 반복 횟수
            cat_features=get_categorical_feature_indices(X_resampled),
            learning_rate=0.1,     # 기본 학습률
            depth=6,               # 트리 깊이 제한
            thread_count=1         # GPU 사용 시 CPU 스레드 최소화
        ))
    ],
    final_estimator=Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(max_iter=1000))
    ]),
    cv=3,
    n_jobs=1   # StackingClassifier 내부의 병렬 처리도 단일 프로세스로 실행
)

# 7. GridSearchCV를 이용한 하이퍼파라미터 튜닝 (ROCAUC 스코어 기준)
param_grid = {
    'final_estimator__lr__C': [0.1, 1.0, 10.0],
    'xgb__max_depth': [4, 6, 8],
    'xgb__learning_rate': [0.01, 0.1, 0.2],
    'lgbm__num_leaves': [31, 63],
    'lgbm__learning_rate': [0.01, 0.1, 0.2],
    'cat__depth': [4, 6, 8],
    'cat__learning_rate': [0.01, 0.1, 0.2]
}

grid_search = GridSearchCV(
    stack_clf,
    param_grid,
    scoring='roc_auc',
    cv=5,      # 교차검증 폴드 수
    n_jobs=-1 # 전체 그리드 탐색은 가능한 모든 코어를 사용
)

# 전체 후보 조합 수 * cv 폴드 수를 총 진행 횟수로 계산
param_combinations = list(ParameterGrid(param_grid))
total_iterations = len(param_combinations) * 5  # cv=5

# tqdm_joblib를 사용해 진행 상황 표시
with tqdm_joblib(tqdm(desc="Grid Search Progress", total=total_iterations, ncols=100)) as progress_bar:
    grid_search.fit(X_train, y_train)

# 검증 데이터셋에 대한 ROC AUC 점수 확인 (선택 사항)
val_pred = grid_search.predict_proba(X_val)[:, 1]
val_roc_auc = roc_auc_score(y_val, val_pred)
print(f'Validation ROC AUC: {val_roc_auc:.4f}')

# 8. 최종 예측 및 제출 파일 생성
pred_proba = grid_search.best_estimator_.predict_proba(X_test_weighted)[:, 1]
submission = pd.DataFrame({
    'ID': [f"TEST_{i:05d}" for i in range(len(test))],
    'probability': pred_proba
})
submission.to_csv('final_weighted_submission.csv', index=False)


Train 데이터 미리보기:
             ID 시술 시기 코드 시술 당시 나이  임신 시도 또는 마지막 임신 경과 연수 시술 유형 특정 시술 유형  \
0  TRAIN_000000   TRZKPL  만18-34세                    NaN   IVF     ICSI   
1  TRAIN_000001   TRYBLT  만45-50세                    NaN   IVF     ICSI   
2  TRAIN_000002   TRVNRY  만18-34세                    NaN   IVF      IVF   
3  TRAIN_000003   TRJXFG  만35-37세                    NaN   IVF     ICSI   
4  TRAIN_000004   TRVNRY  만18-34세                    NaN   IVF     ICSI   

   배란 자극 여부    배란 유도 유형  단일 배아 이식 여부  착상 전 유전 검사 사용 여부  ...  배아 이식 경과일  \
0         1  기록되지 않은 시행          0.0               NaN  ...        3.0   
1         0      알 수 없음          0.0               NaN  ...        NaN   
2         1  기록되지 않은 시행          0.0               NaN  ...        2.0   
3         1  기록되지 않은 시행          0.0               NaN  ...        NaN   
4         1  기록되지 않은 시행          0.0               NaN  ...        3.0   

   배아 해동 경과일  임신 성공 여부  고위험 시술 여부  단일 배아 성공 가능성  불임 주요 원인 개수  IVF 성공률  \
0        NaN   

KeyError: "테스트 데이터에 '임신 성공 여부' 열이 존재하지 않습니다."